# Inverse Problems and Parameter Estimation, GEOS 627/427, University of Alaska Fairbanks

- hwsol_cov2D.ipynb

In [ ]:
%matplotlib inline
#%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from lib_fft import *
from lib_inverse import covC

import warnings
warnings.filterwarnings('ignore')

In [ ]:
run_xy2distance = True

if run_xy2distance:
    bdisplay = True
    #nx = 3; ny = 5      #  9 5 x 5 Toeplitz blocks
    nx = 5; ny = 3      # 25 3 x 3 Toeplitz blocks
    xmin = -4
    xmax = 12
    Dx = xmax - xmin
    dx = Dx/(nx-1)
    [iD,ix0,iy0,ix,iy,PA,PB] = xy2distance(nx,ny,bdisplay); 
    #whos ix0 iy0 iD ix iy PA PB

In [ ]:
if run_xy2distance:
    ymin = -1.5
    xvec = xmin + ix0*dx
    yvec = ymin + iy0*dx
    X,Y = np.meshgrid(xvec,yvec)
    D = dx * iD
    # check values
    print('xvec: %i pts from %.2f to %.2f' % (len(xvec),min(xvec),max(xvec)))
    print('yvec: %i pts from %.2f to %.2f' % (len(yvec),min(yvec),max(yvec)))
    
    raise SystemExit("stop here")

In [ ]:
# USER INPUT: n, Lprime (=iLprime*dx), sigma

# discretize the grid
# (assume that x and y are in units of km)
npower = 5                # default 5; 7 for Problem 1-7,1-8
nx = int(2**npower)
ny = int(nx/2)
xmin = -10
xmax = 108
ymin = -20
#nx = 2**5; ny = nx;      # square grid

# correlation length of 2D Gaussian fields
# NOTE: actual correlation length is Lprime = iLprime*dx
iLprime = 0.1250*nx

# standard deviation of 2D fields
sigma = 0.05

ichol = 1         # (0,1) generate samples using Cholesky algorithm
ifourier = 0      # (0,1) generate samples using FFT algorithm
idouble = 0       # (0,1) =1 to remove periodic boundaries (ifourier=1 only)
icov = 1          # (1,2,3,4) type of covariance matrix (see stcovs)

# number of samples of the covariance matrix to generate
nsample = 1000

# max n to allow for computing the n x n covariance matrix
NMAX = 2**10

In [ ]:
n = nx*ny      # dimension of covariance matrix
stcovs = ['Gaussian','Exponential','Circular','Matern']
stcov = stcovs[icov-1]

# uniform increment in both directions (dy = dx)
Dx = xmax - xmin
dx = Dx/(nx-1)

# grid for plotting
[iDrow1,ix0,iy0] = xy2distance_row1(nx,ny)
ymax = ymin + dx*(ny-1)
xvec = xmin + ix0*dx
yvec = ymin + iy0*dx
[X,Y] = np.meshgrid(xvec,yvec)

# max distance between two gridpoints
# (hypotenuse of the rectangular grid)
xran = xmax-xmin
yran = ymax-ymin
dmax = np.sqrt( xran**2 + yran**2 )

# correlation length of 2D Gaussian fields
Lprime = iLprime*dx

In [ ]:
# display output

# check
idmax = np.sqrt((nx-1)**2 + (ny-1)**2)

print('dx       = %.2f' % dx)
print('nx       = %i' % nx)
print('ny       = %i' % ny)
print('dmax     = %.2f' % dmax)
print('idmax    = %.2f' % idmax)
print('dx*idmax = %.2f' % (dx*idmax))
print('iLprime  = %.2f' % iLprime)
print('Lprime   = iLprime*dx = %.2f' % Lprime)

stit0 = 'Lprime = %.2f, sigma = %.2f' % (Lprime,sigma)
stit1 = '(nx,ny) = (%i,%i), Lprime = %.2f, \\sigma = %.2f' % (nx,ny,Lprime,sigma)
stit2 = 'Block Toeplitz: %i x %i (%i) blocks, each %i x %i' % (nx,nx,nx*nx,ny,ny)
print()
print(stit0)
print(stit1)
print(stit2)

In [ ]:
print('nx       = %i' % nx)
print('ny       = %i' % ny)
print('n        = %i' % n)
print('NMAX     = %i' % n)

In [ ]:
# plot covariance function C(d) using a finer discretization
dmaxt = 5*Lprime     # for plotting
dcont = np.linspace(0,dmaxt,500)
# note: input is in distances (dcont,L)
ccont = covC(dcont,[icov,Lprime,sigma])

In [ ]:
if n <= NMAX:
    # generate mesh of points
    iD,ix0,iy0,_,_,_,_ = xy2distance(nx,ny)
    ymax = ymin + dx*(ny-1)

    # note: input is in indices (iD,iLprime)
    C = covC(iD,[icov,iLprime,sigma])

    plt.figure()
    axtemp = [0,dmaxt,sigma**2*(-0.1),sigma**2*1.1 ]
    plt.plot(axtemp[:2],[0,0],'k')
    plt.plot([Lprime,Lprime],axtemp[2:4],'r')
    plt.plot(axtemp[:2],covC(Lprime,[icov,Lprime,sigma])*np.array([1,1]),'r')
    plt.plot(dcont,ccont,'b')
    plt.plot(iD*dx,C,'b.')
    plt.xlabel('Distance')
    plt.axis(axtemp)
    plt.title('%s covariance with Lprime = %.2f' %(stcov,Lprime))
    # vector associated with parameterization
    Atot = nx*ny*dx**2              # NOTE: does not equal xran*yran
    dA = Atot/n
    dAvec = np.ones((n,1))*dA
    Avec = np.sqrt( dAvec/Atot )
    #n*dA - Atot, sum(Avec.^2)         % check

    # inverse covariance matrix for norm operations
    Cmodinv  = np.diag(Avec) * np.linalg.inv(C) * np.diag(Avec)       # unstable for high cond(C)
    Cdiaginv = np.diag(Avec) * np.diag(1/np.diag(C)) * np.diag(Avec)  # approximation

    # initial plots

    plt.figure(figsize=(7,8))
    nr = 2
    nc = 1
    plt.subplot(nr,nc,1)
    plt.plot(X,Y,'b.')
    #plt.axis(ax1aex)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Plotting grid (nx, ny, dx) = (%i, %i, %.2f), (x0, y0) = (%.1f, %.1f)' %(nx,ny,dx,xmin,ymin))
    plt.subplot(nr,nc,2)
    plt.plot((X-xmin)/dx,(Y-ymin)/dx,'r.')
    #axis(ax1bex-1)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Computational grid (nx, ny) = (%i, %i)' %(nx,ny))
    plt.subplots_adjust(hspace=0.25)

    # plot (1) theoretical mean and (2) theoretical covariance matrix
    plt.figure(figsize=(7,8))
    nr = 2
    nc = 1
    plt.subplot(nr,nc,1)
    plt.imshow(np.zeros((n,1)).reshape(ny,nx,order='F'))
    plt.xlabel('x (unshifted and unscaled)')
    plt.ylabel('y (unshifted and unscaled)')
    plt.title('theoretical mean')
    #caxis(sigma*[-1 1])
    #colorbar
    #axis(ax1bex)

    plt.subplot(nr,nc,2)
    plt.imshow(C)
    #plt.title({sprintf('C: %s covariance matrix with %i^2 = (%i*%i)^2 entries',stcov,n,nx,ny),stit2})
    #axis equal, axis(axcex)
    #caxis([0 sigma^2])
    plt.colorbar()
    
    # plot distance matrix that the covariance matrix is based on
    plt.figure()
    plt.imshow(dx*iD)
    plt.title(('dx*iD: distance matrix with %i^2 = (%i*%i)^2 entries \n' %(n,nx,ny))+stit2)
    # axis equal, axis(axcex)
    # caxis([0 dmax])
    plt.colorbar()
    
    # plot histogram of distances among gridpoints
    plt.figure()
    plt.hist(dx*iD.flatten(order='F'),range(0,140,4))
    plt.xlabel('Distance between gridpoints, km')

In [ ]:
# define an overarching function
def run_cov(ichol,ifourier,idouble,nx,ny,icov,iLprime,sigma):
    print('ichol = %i, ifourier = %i, idouble = %i' % (ichol,ifourier,idouble))
    print('icov = %i, iLprime = %i, sigma = %.2f' % (icov,iLprime,sigma))
    print('nx = %i, ny = %i' % (nx,ny))
    
    # initialize samples
    covm_samples = np.empty((n,nsample))

    # in general, one can avoid filling up matrix entries when using the Fourier method;
    # here we calculate them for plotting purposes
    iD,ix0,iy0,_,_,_,_ = xy2distance(nx,ny)
    C = covC(iD,[icov,iLprime,sigma])
    
    if ichol==1:
        print('CHOLESKY ALGORITHM')
        # generate samples of the prior
        R = np.linalg.cholesky(C)
        covm_samples = np.empty((n,nsample))
        for ii in range(nsample):
            covm_samples[:,ii] = R @ np.random.randn(n)
            
    if ifourier==1:
        print('FFT ALGORITHM')
        if idouble==1:
            nxd = 2*nx
            nyd = 2*ny
        else:
            nxd = nx
            nyd = ny
        ix0d = np.arange(nxd)
        iy0d = np.arange(nyd)
        # discretization in wavenumber space
        xs   = ix0d*dx
        ys   = iy0d*dx
        kvec = k_of_x(xs)
        lvec = k_of_x(ys)
        [Kmat,Lmat] = np.meshgrid(kvec,lvec)

        # covariance function is the first row (or column)
        # note that nxd and nyd may not equal nx and ny
        #c = C(1,:)
        iDrow1,ix0dum,iy0dum = xy2distance_row1(nxd,nyd)
        c = covC(iDrow1,[icov,iLprime,sigma])

        # preparing for the 2D FFT
        cmod = c.reshape(nyd,nxd,order='F')
        cmod[int(nyd/2)+1:,:] = np.flipud(cmod[1:int(nyd/2),:])
        cmod[:,int(nxd/2)+1:] = np.fliplr(cmod[:,1:int(nxd/2)])
        #figure; imagesc(cmod); axis equal; axis tight;

        # 2D FFT
        # general case (using the first row of the covariance matrix)
        [kvec, lvec, CKL0] = mhfft2(xs,ys,cmod)
        CKL = np.real(CKL0)
        # check that Cfunk has nominal imaginary component
        #norm(real(CKL0)), norm(imag(CKL0))
        # compute samples

        # samples of a 2D Gaussian random field (GRF)
        phi_h,Adum,Bdum = grf2(kvec,lvec,2*CKL,nsample)   # use 2*Cfun since will take real part later
        [xss,yss,phi] = mhifft2(kvec,lvec,phi_h,rflag=0)  # inverse 2D FFT
        covm_samples = np.empty((n,nsample))
        for ii in range(nsample):
            covm_samples[:,ii] = np.real(phi[:ny, :nx, ii]).flatten(order='F').reshape(n, order='F')
            
    #======================================================================
    # START YOUR PLOTS AND CALCULATIONS HERE
    # (You will also need to change the USER INPUT at the top of the code.)
      
    # plot samples
    plt.figure(figsize=(14,15))
    ax1 = plt.gca()
    ax1.set_aspect('equal',adjustable='box')
    nc = 2
    nr = 4
    for ii in range(nc*nr):
        plt.subplot(nr,nc,ii+1)  
        plt.pcolormesh(X,Y,covm_samples[:,ii].reshape(ny,nx,order='F'))  
        plt.clim(3*sigma*np.array([-1,1]))
        plt.title('%s sample %i/%i' % (stcov,ii,nsample))
    #plt.subplots_adjust(hspace=0.35)
    
    # PROBLEM 1-3
    # compute and plot the sample mean
    # (use the code block at the bottom)


    # PROBLEM 1-4
    # compute and plot covariance matrix from the samples
    # (use the code block at the bottom)

  

    if ifourier==1:
        # PROBLEM 1-8
        # vary icov and see how the GRFs change (ifourier = 1)
        # (code extracted from above)
        icvec = np.array([1,2,3])
        #icvec = np.array([1,2])
        nrun = len(icvec)
        nsamp = 8                 # choose small subset
        for xx in range(nrun):    # loop over different covariance functions
            icov = icvec[xx]
            c = covC(iDrow1,[icov,iLprime,sigma])
            cmod = c.reshape(int(nyd),int(nxd),order='F')
            cmod[int(nyd/2)+1:,:] = np.flipud(cmod[1:int(nyd/2),:])
            cmod[:,int(nxd/2)+1:] = np.fliplr(cmod[:,1:int(nxd/2)])
            kvec,lvec,CKL0 = mhfft2(xs,ys,cmod)
            CKL = np.real(CKL0)
            
            # REPLACE THE FOLLOWING LINE WITH OTHER LINES (see grf2())
            phi_h,_,_ = grf2(kvec,lvec,2*CKL,nsamp)
            

            xss,yss,phi = mhifft2(kvec,lvec,phi_h,rflag=0)

            csamples = np.real(phi[:ny,:nx,0]).flatten(order='F')
            for ii in range(1,nsamp):
                # take only the nx x ny patch, then convert to a column vector
                ptemp = np.real(phi[:ny,:nx,ii]).flatten(order='F')
                csamples = np.vstack((csamples,ptemp))
            csamples = csamples.T
            
            if False:    # plot 8 samples
                plt.figure(figsize=(12,14)) 
                nc = 2
                nr = 4
                for ii in range(nc*nr):
                    plt.subplot(nr,nc,ii+1)
                    plt.pcolormesh(X,Y,csamples[:,ii].reshape(ny,nx,order='F'))
                    plt.clim(3*sigma*np.array([-1,1]))
                    #plt.title(r'$%s  covariance , Lprime = %.2f, \sigma = %.2f$' %(stcovs[int(icvec[xx])-1],Lprime,sigma))
                    plt.title(r'%s covariance, Lprime = %.2f, $\sigma$ = %.2f' %(stcovs[int(icvec[xx])-1],Lprime,sigma))
                    plt.axis('scaled')
                
            # 3 x 1 subplot will take the first of the 8 samples
            # each row will be a different covariance function
            # (note: this figsize is not used)
            plt.figure(num=99,figsize=(8.5,10.5))
            plt.subplot(3,1,xx+1)
            plt.pcolormesh(X,Y,csamples[:,0].reshape(ny,nx,order='F'))
            plt.clim(3*sigma*np.array([-1,1]))
            #plt.title(r'$%s  covariance , Lprime = %.2f, \sigma = %.2f$' %(stcovs[int(icvec[xx])-1],Lprime,sigma))
            plt.title(r'%s covariance, Lprime = %.2f, $\sigma$ = %.2f' % (stcovs[int(icvec[xx])-1],Lprime,sigma))
            plt.subplots_adjust(hspace=0.35)
            plt.axis('scaled')

    return covm_samples

In [ ]:
# use Cholesky matrix square root
ichol = 1
ifourier = 0
idouble = 0
covm_samples = run_cov(ichol,ifourier,idouble,nx,ny,icov,iLprime,sigma)

In [ ]:
# PROBLEM 1-3
# compute and plot the sample mean
# (when you get the code working, copy it into the wrapper function above)





In [ ]:
# PROBLEM 1-4
# compute and plot covariance matrix from the samples
# (when you get the code working, copy it into the wrapper function above)


